In [ ]:
import copy
import os

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch

from fair import FAIR
from fair.io import read_properties
from fair.interface import initialise, fill

In [ ]:
os.makedirs('../plots', exist_ok=True)

In [ ]:
f = FAIR()

In [ ]:
f.define_time(1750, 2100, 1)

In [ ]:
f.define_scenarios(['ssp126'])

In [ ]:
pd.options.display.max_columns = 50
fair_params_df = pd.read_csv('../data/fair2.1-parameters/ar6_calibration_ebm3.csv', index_col=0)
fair_params_df

In [ ]:
species, properties = read_properties()

In [ ]:
f.define_species(species, properties)

In [ ]:
[fair_params_df.index[10]]

In [ ]:
f.define_configs([fair_params_df.index[10]])

In [ ]:
f.allocate()

In [ ]:
f.fill_from_rcmip()

In [ ]:
ar6_forcing_file = pooch.retrieve(
    url="doi:10.5281/zenodo.5705391/table_A3.3_historical_ERF_1750-2019_best_estimate.csv",
    known_hash="md5:f64915777d2971c5eb5d96a432f45c48",
)

In [ ]:
ar6_forcing_df = pd.read_csv(ar6_forcing_file, index_col=0)

In [ ]:
volcanic_forcing = np.zeros(351)
volcanic_forcing[:270] = ar6_forcing_df['volcanic'].values
volcanic_forcing[269:281] = np.linspace(1, 0, 12) * volcanic_forcing[269]

In [ ]:
solar_forcing=np.zeros(351)
solar_forcing_df = pd.read_csv('../data/forcing/solar_erf.csv', index_col=0)
solar_forcing = solar_forcing_df['solar_erf'].values[:351]

In [ ]:
# Volcanic forcing
fill(f.forcing, volcanic_forcing[:, None, None] * fair_params_df.loc[fair_params_df.index[0], 'scale Volcanic'], specie='Volcanic')

# Solar forcing
trend_shape = np.ones(351)
trend_shape[:271] = np.linspace(0, 1, 271)

fill(f.forcing, 
     solar_forcing[:, None, None] * 
     fair_params_df.loc[fair_params_df.index[0], 'solar_amplitude'] + 
     trend_shape[:, None, None] * fair_params_df.loc[fair_params_df.index[0], 'solar_trend'],
     specie='Solar'
)

In [ ]:
f.fill_species_configs()

In [ ]:
# Climate response
fill(f.climate_configs['ocean_heat_capacity'], fair_params_df.loc[fair_params_df.index[10],'c1':'c3'])
fill(f.climate_configs['ocean_heat_transfer'], fair_params_df.loc[fair_params_df.index[10],'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[fair_params_df.index[10],'epsilon'])
fill(f.climate_configs['sigma_eta'], fair_params_df.loc[fair_params_df.index[10],'sigma_eta'])
fill(f.climate_configs['sigma_xi'], fair_params_df.loc[fair_params_df.index[10],'sigma_xi'])
fill(f.climate_configs['gamma_autocorrelation'], fair_params_df.loc[fair_params_df.index[10],'gamma'])
fill(f.climate_configs['seed'], fair_params_df.index[10]*399 + 1355763)
fill(f.climate_configs['use_seed'], True)
fill(f.climate_configs['stochastic_run'], True)

# carbon cycle
fill(f.species_configs['iirf_0'], fair_params_df.loc[fair_params_df.index[10], 'r0'], specie='CO2')
fill(f.species_configs['iirf_airborne'], fair_params_df.loc[fair_params_df.index[10], 'rA'], specie='CO2')
fill(f.species_configs['iirf_uptake'], fair_params_df.loc[fair_params_df.index[10], 'rU'], specie='CO2')
fill(f.species_configs['iirf_temperature'], fair_params_df.loc[fair_params_df.index[10], 'rT'], specie='CO2')

# aerosol direct
for specie in ['BC', 'CH4', 'N2O', 'NH3', 'NOx', 'OC', 'Sulfur', 'VOC', 'Equivalent effective stratospheric chlorine']:
    fill(f.species_configs['erfari_radiative_efficiency'], fair_params_df.loc[fair_params_df.index[10], f'ari {specie}'], specie=specie)

# aerosol indirect
fill(f.species_configs['aci_scale'], fair_params_df.loc[fair_params_df.index[10], 'beta'])
fill(f.species_configs['aci_shape'], fair_params_df.loc[fair_params_df.index[10], 'shape_so2'], specie='Sulfur')
fill(f.species_configs['aci_shape'], fair_params_df.loc[fair_params_df.index[10], 'shape_bc'], specie='BC')
fill(f.species_configs['aci_shape'], fair_params_df.loc[fair_params_df.index[10], 'shape_oc'], specie='OC')

# ozone
for specie in ['CH4', 'N2O', 'Equivalent effective stratospheric chlorine', 'CO', 'VOC', 'NOx']:
    fill(f.species_configs['ozone_radiative_efficiency'], fair_params_df.loc[fair_params_df.index[10], f'o3 {specie}'], specie=specie)

# methane lifetime baseline
fill(f.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people might wanna run pulse expts with these gases)
fill(f.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
fill(f.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

# forcing scaling

for specie in ['CH4', 'N2O', 'Stratospheric water vapour', 'Contrails', 'Light absorbing particles on snow and ice', 'Land use']:
    fill(f.species_configs['forcing_scale'], fair_params_df.loc[fair_params_df.index[10], f'scale {specie}'], specie=specie)
for specie in ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']:
    fill(f.species_configs['forcing_scale'], fair_params_df.loc[fair_params_df.index[10], 'scale minorGHG'], specie=specie)

calibrated_f4co2_mean = 7.866801427264765
fill(f.species_configs['forcing_scale'], 1 + 0.561*(calibrated_f4co2_mean - fair_params_df.loc[fair_params_df.index[10],'F_4xCO2'])/calibrated_f4co2_mean, specie='CO2')


# tune down volcanic efficacy
fill(f.species_configs['forcing_efficacy'], 0.6, specie='Volcanic')


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f.species_configs['baseline_concentration'], fair_params_df.loc[fair_params_df.index[10], 'co2_concentration_1750'], specie='CO2')

f.ch4_method='Thornhill2021'

In [ ]:
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

In [ ]:
f.run()

In [ ]:
g = copy.deepcopy(f)
fill(g.concentration, np.nan)
initialise(g.concentration, g.species_configs['baseline_concentration'])
initialise(g.forcing, 0)
initialise(g.temperature, 0)
initialise(g.airborne_emissions, 0)
initialise(g.cumulative_emissions, 0)
initialise(g.alpha_lifetime, 1)
initialise(g.gas_partitions, np.array([0, 0, 0, 0]))

g.emissions[272:280, 0, 0, 3] = np.linspace(f.emissions[272, 0, 0, 3].values, 0, 8)
g.emissions[280:, 0, 0, 3] = 0
g.run()

In [ ]:
fig, ax = pl.subplots(1, 2, squeeze=True)
ax[0].plot(f.timepoints, f.emissions[:, 0, :, 3]);
ax[0].plot(g.timepoints, g.emissions[:, 0, :, 3]);
ax[0].axhline(0, color='k', lw=0.5, ls=':')
ax[0].set_xlim(1850, 2100)
ax[0].set_ylim(-50, 400)
ax[0].set_title("Net CH$_4$ emissions")
ax[0].set_ylabel("MtCH$_4$ yr$^{-1}$")

ax[1].plot(f.timebounds, f.temperature[:, 0, :, 0] - f.temperature[100:151, 0, :, 0].mean(axis=0), label='SSP1-2.6');
ax[1].plot(g.timebounds, g.temperature[:, 0, :, 0] - g.temperature[100:151, 0, :, 0].mean(axis=0), label='Methane removal');
ax[1].axhline(1.5, color='r', lw=0.5, ls=':')
ax[1].set_xlim(1850, 2100)
ax[1].set_ylim(-.5, 2.5)
ax[1].set_title("Global mean temperature anomaly")
ax[1].set_ylabel("K")
ax[1].legend()

fig.tight_layout()
pl.savefig(f'../plots/emis_temp_ens{fair_params_df.index[10]}.png')

In [ ]:
np.sum(g.emissions[:, 0, 0, 3] - f.emissions[:, 0, 0, 3])

In [ ]:
fig, ax = pl.subplots(1, 2, squeeze=True)
ax[0].plot(f.timebounds, f.concentration[:, 0, :, 3]);
ax[0].plot(g.timebounds, g.concentration[:, 0, :, 3]);
ax[0].axhline(0, color='k', lw=0.5, ls=':')
ax[0].set_xlim(1850, 2100)
ax[0].set_ylim(600, 2000)
ax[0].set_title("CH$_4$ concentration")
ax[0].set_ylabel("ppb")

ax[1].plot(f.timebounds, f.alpha_lifetime[:, 0, :, 3] * 10.4198121, label='SSP1-2.6');
ax[1].plot(g.timebounds, g.alpha_lifetime[:, 0, :, 3] * 10.4198121, label='Methane removal');
ax[1].set_xlim(1850, 2100)
ax[1].set_ylim(8, 12)
ax[1].set_title("Methane lifetime")
ax[1].set_ylabel("yr")
ax[1].legend()

fig.tight_layout()
pl.savefig(f'../plots/conc_lifetime_ens{fair_params_df.index[10]}.png')

In [ ]:
fig, ax = pl.subplots(1, 2, squeeze=True)
ax[0].plot(f.timebounds, f.forcing[:, 0, :, 57]);
ax[0].plot(g.timebounds, g.forcing[:, 0, :, 57]);
ax[0].axhline(0, color='k', lw=0.5, ls=':')
ax[0].set_xlim(1850, 2100)
ax[0].set_ylim(-0.05, 0.35)
ax[0].set_title("Ozone forcing")
ax[0].set_ylabel("W m$^{-2}$")

ax[1].plot(f.timebounds, f.forcing[:, 0, :, 60], label='SSP1-2.6');
ax[1].plot(g.timebounds, g.forcing[:, 0, :, 60], label='Methane removal');
ax[1].axhline(0, color='k', lw=0.5, ls=':')
ax[1].set_xlim(1850, 2100)
ax[1].set_ylim(-0.02, 0.16)
ax[1].set_title("Strat. water vapour forcing")
ax[1].set_ylabel("W m$^{-2}$")
ax[1].legend()

fig.tight_layout()
pl.savefig(f'../plots/ozone_h2o_ens{fair_params_df.index[10]}.png')

In [ ]:
f.forcing

In [ ]:
fig, ax = pl.subplots()
ax.plot(f.timebounds, f.forcing_sum[:, 0, :], label='SSP1-2.6');
ax.plot(g.timebounds, g.forcing_sum[:, 0, :], label='Methane removal');
ax.set_xlim(1850, 2100)
ax.set_ylim(-2, 4)
ax.set_title("Total effective radiative forcing")
ax.set_ylabel("W m$^{-2}$")
ax.axhline(0, color='k', lw=0.5, ls=':')
ax.legend()

fig.tight_layout()
pl.savefig(f'../plots/forcing_ens{fair_params_df.index[10]}.png')

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Ozone', config=fair_params_df.index[10], scenario='ssp126')])
pl.plot(g.timebounds, g.forcing.loc[dict(specie='Ozone', config=fair_params_df.index[10], scenario='ssp126')])

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Aerosol-radiation interactions', config=fair_params_df.index[10], scenario='ssp126')])
pl.plot(g.timebounds, g.forcing.loc[dict(specie='Aerosol-radiation interactions', config=fair_params_df.index[10], scenario='ssp126')])

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Stratospheric water vapour', config=fair_params_df.index[10], scenario='ssp126')])
pl.plot(g.timebounds, g.forcing.loc[dict(specie='Stratospheric water vapour', config=fair_params_df.index[10], scenario='ssp126')])